In [18]:
# %%
import os
import requests
import pandas as pd
import time

In [19]:
# %%
url = "https://www.nndc.bnl.gov/radlist/radlist2.jsp"
inputDirectory = "../../../Downloads/ENSDF-230601"

## ENSDF to DataFrames

In [20]:
# %%

format = "csv" #"json" or "csv"
outputDirectory = f"../../../Downloads/ensdf_output_{format}"


for fileName in os.listdir(inputDirectory):
    file = os.path.join(inputDirectory, fileName)
    if os.path.isfile(file):
        with open(file, "r") as decayFile:
            mydata = decayFile.read()
            chunks = mydata.split("\n\n")
            a = os.path.splitext(fileName)[1][1:] # Gets the mass number 'a' from the ensdf file name by extracting it from the extension
            z = 0 # not the real atomic number.
            print("Working on", a)
            for ensdf in chunks:
                response = requests.post(url, data={"mydata": ensdf, "format": format})
                time.sleep(2)
                outputPath = os.path.join(outputDirectory, str(a) + "_" + str(z))
                if "<html>" in response.text:
                    continue # filters the non-ENSDF segments in the files since radlist returns "Please check input" for them.
                with open(f"{outputPath}.{format}", "w") as f:
                    f.write(response.text)
                z += 1

Working on 289
Working on 270
Working on 203
Working on 092
Working on 262
Working on 232
Working on 091


ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

## Creating an HDF Dataset

### From JSoN 

In [116]:
import json

In [127]:
outputDirectory = "../../../Downloads/ensdf_output"

all_data = [] # list of dataframes of radiation data

for fileName in os.listdir(outputDirectory):
    file = os.path.join(outputDirectory, fileName)
    if os.path.splitext(file)[1] == ".json":
        with open(file) as file_object:
            result = json.load(file_object)
            rad_data = result['radiation']
            parent_data = result['parent']
            if len(rad_data)>0:
                # creating list of dataframes for different radiations.
                rad_list = [pd.DataFrame(rad_data[v]) for v in rad_data.keys()]
                
                df = pd.concat(rad_list,keys = list(rad_data.keys()))
                df["Isotope"] = parent_data["element"] + parent_data["a"]

                df["Element"] = parent_data["element"]
                all_data.append(df)
            else:
                continue
                # Omitting those nuclides from the dataset that have no radiation data

# TODO: add download_timestamp, format heading, discuss adding subtype, 

In [128]:
total_data = pd.concat(all_data)

# %%
total_data.groupby("Element")

total_data["Radiation"] = total_data.index.get_level_values(0)
total_data.set_index('Isotope', inplace=True)

pd.set_option("display.max_rows", None, "display.max_columns", None, 'display.width', None)

In [129]:
total_data.drop(['dose','type'], axis=1, inplace=True)

total_data.to_hdf("compiled_ensdf_csv.hdf", key="decay_data")

total_data.loc["Pa230"]

,radIntensityUncertainty,radIntensity,energyUncertainty,subType,epEnergyUncertainty,energy,epEnergy,doseUncertainty,Element,Radiation
Isotope,,,,,,,,,,
Pa230,3,6.E-6,2,,,4766,,15,Pa,alpha
Pa230,10,9.6E-7,5,,,4798,,5,Pa,alpha
Pa230,6,1.3E-5,3,,,4934,,3,Pa,alpha
Pa230,6,2.2E-5,2,,,4973,,3,Pa,alpha
Pa230,6,1.3E-5,3,,,5060,,3,Pa,alpha
Pa230,6,2.2E-5,2,,,5084,,3,Pa,alpha
Pa230,6,1.9E-5,3,,,5119,,3,Pa,alpha
Pa230,3,1.3E-5,2,,,5153,,17,Pa,alpha
Pa230,6,1.6E-5,3,,,5183,,3,Pa,alpha


### From CSV

In [123]:
outputDirectory = "../../../Downloads/ensdf_output_csv"

all_data = [] 
for fileName in os.listdir(outputDirectory):
    file = os.path.join(outputDirectory, fileName)
    if os.path.splitext(file)[1] == ".csv":
        data = pd.read_csv(
            file, 
        ) # converts every csv file to Dataframe and appends it to all_data
        all_data.append(data)
        if data.A.dtype == object:
            print(file)

In [107]:
total_data = pd.concat(all_data)

In [112]:
# %%
total_data.groupby("Element")

total_data["Isotope"] = total_data.Element.map(str) + total_data.A.map(str) # for csv only
# can make isotopes as indices like tardis-nuclear.

# %%
total_data.to_hdf("compiled_ensdf_csv.hdf", key="decay_data")

/home/syzygy/miniconda3/envs/tardis/lib/python3.8/site-packages/pandas/core/generic.py:2490: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->Index(['Element', 'Parent E(level)', 'Uncertainty', 'JPi', 'Decay Mode',
       'Q Value Uncertainty', 'T1/2 (txt)', 'Gammas Balance', 'X-Rays Balance',
       'B- Balance', 'B+ Balance', 'Conversion Electrons Balance',
       'Auger Electrons Balance', 'Neutrinos Balance', 'Recoil Balance',
       'Alphas Balance', 'Sum Balance', 'Q-effective Balance',
       'Missing Energy Balance', 'Ratio Balance', 'Daughter', 'Radiation',
       'Rad subtype', 'Uncertainty.1', 'Uncertainty.3', 'Uncertainty.4',
       'Isotope'],
      dtype='object')]

  pytables.to_hdf(


In [114]:
total_data[total_data.Isotope == "Rb92"]

,A,Element,Z,N,Parent E(level),Uncertainty,JPi,Decay Mode,Decay Mode Value,Decay Mode Uncertainty,Q Value,Q Value Uncertainty,T1/2 (txt),T1/2 (sec),Gammas Balance,X-Rays Balance,B- Balance,B+ Balance,Conversion Electrons Balance,Auger Electrons Balance,Neutrinos Balance,Recoil Balance,Neutrons Balance,Protons Balance,Alphas Balance,Sum Balance,Q-effective Balance,Missing Energy Balance,Ratio Balance,Daughter,Radiation,Rad subtype,Rad Energy,Uncertainty.1,EP Energy,Uncertainty.2,Rad Intensity,Uncertainty.3,Dose,Uncertainty.4,Isotope
0,92,Rb,37,55,0,NaN,0-,B-,100.0,NaN,8095.0,6,4.48 S 3,4.48,170 9,4.7E-4 3,3.64E+3 3,0,0.039 3,2.72E-4 17,4.29E+3 3,0,0,0,0,8.10E+3 5,8095 6,0 AP,0 % AP,92Sr,g,XR l,1.810,NaN,NaN,NaN,1.490000e-04,12,2.700000e-09,22,Rb92
1,92,Rb,37,55,0,NaN,0-,B-,100.0,NaN,8095.0,6,4.48 S 3,4.48,170 9,4.7E-4 3,3.64E+3 3,0,0.039 3,2.72E-4 17,4.29E+3 3,0,0,0,0,8.10E+3 5,8095 6,0 AP,0 % AP,92Sr,g,XR ka2,14.098,NaN,NaN,NaN,9.500000e-04,9,1.350000e-07,12,Rb92
2,92,Rb,37,55,0,NaN,0-,B-,100.0,NaN,8095.0,6,4.48 S 3,4.48,170 9,4.7E-4 3,3.64E+3 3,0,0.039 3,2.72E-4 17,4.29E+3 3,0,0,0,0,8.10E+3 5,8095 6,0 AP,0 % AP,92Sr,g,XR ka1,14.165,NaN,NaN,NaN,1.840000e-03,17,2.600000e-07,24,Rb92
3,92,Rb,37,55,0,NaN,0-,B-,100.0,NaN,8095.0,6,4.48 S 3,4.48,170 9,4.7E-4 3,3.64E+3 3,0,0.039 3,2.72E-4 17,4.29E+3 3,0,0,0,0,8.10E+3 5,8095 6,0 AP,0 % AP,92Sr,g,XR kb3,15.825,NaN,NaN,NaN,1.370000e-04,13,2.170000e-08,20,Rb92
4,92,Rb,37,55,0,NaN,0-,B-,100.0,NaN,8095.0,6,4.48 S 3,4.48,170 9,4.7E-4 3,3.64E+3 3,0,0.039 3,2.72E-4 17,4.29E+3 3,0,0,0,0,8.10E+3 5,8095 6,0 AP,0 % AP,92Sr,g,XR kb1,15.836,NaN,NaN,NaN,2.650000e-04,25,4.200000e-08,4,Rb92
5,92,Rb,37,55,0,NaN,0-,B-,100.0,NaN,8095.0,6,4.48 S 3,4.48,170 9,4.7E-4 3,3.64E+3 3,0,0.039 3,2.72E-4 17,4.29E+3 3,0,0,0,0,8.10E+3 5,8095 6,0 AP,0 % AP,92Sr,g,XR kb2,16.085,NaN,NaN,NaN,4.700000e-05,4,7.600000e-09,7,Rb92
6,92,Rb,37,55,0,NaN,0-,B-,100.0,NaN,8095.0,6,4.48 S 3,4.48,170 9,4.7E-4 3,3.64E+3 3,0,0.039 3,2.72E-4 17,4.29E+3 3,0,0,0,0,8.10E+3 5,8095 6,0 AP,0 % AP,92Sr,g,NaN,386.100,3,NaN,NaN,2.400000e-02,5,9.400000e-05,20,Rb92
7,92,Rb,37,55,0,NaN,0-,B-,100.0,NaN,8095.0,6,4.48 S 3,4.48,170 9,4.7E-4 3,3.64E+3 3,0,0.039 3,2.72E-4 17,4.29E+3 3,0,0,0,0,8.10E+3 5,8095 6,0 AP,0 % AP,92Sr,g,NaN,393.500,1,NaN,NaN,1.220000e-01,16,4.800000e-04,6,Rb92
8,92,Rb,37,55,0,NaN,0-,B-,100.0,NaN,8095.0,6,4.48 S 3,4.48,170 9,4.7E-4 3,3.64E+3 3,0,0.039 3,2.72E-4 17,4.29E+3 3,0,0,0,0,8.10E+3 5,8095 6,0 AP,0 % AP,92Sr,g,NaN,569.800,1,NaN,NaN,5.400000e-01,8,3.100000e-03,4,Rb92
9,92,Rb,37,55,0,NaN,0-,B-,100.0,NaN,8095.0,6,4.48 S 3,4.48,170 9,4.7E-4 3,3.64E+3 3,0,0.039 3,2.72E-4 17,4.29E+3 3,0,0,0,0,8.10E+3 5,8095 6,0 AP,0 % AP,92Sr,g,NaN,703.600,3,NaN,NaN,4.500000e-02,11,3.200000e-04,8,Rb92


In [115]:
# %%
total_data[(total_data.Isotope == "Rb92") & (total_data.Radiation == "g")]["Rad Intensity"]

0     0.000149
1     0.000950
2     0.001840
3     0.000137
4     0.000265
5     0.000047
6     0.024000
7     0.122000
8     0.540000
9     0.045000
10    0.109000
11    3.200000
12    0.147000
13    0.012800
14    0.035000
15    0.096000
16    0.134000
17    0.350000
18    0.051000
19    0.038000
20    0.420000
21    0.035000
22    0.058000
23    0.054000
24    0.067000
25    0.070000
26    0.077000
27    0.600000
28    0.030000
29    0.070000
30    0.100000
31    0.035000
32    0.042000
33    0.035000
34    0.032000
35    0.061000
36    0.210000
37    0.100000
38    0.100000
39    0.106000
40    0.080000
41    0.240000
42    0.110000
43    0.080000
44    0.160000
45    0.070000
46    0.090000
47    0.058000
48    0.077000
49    0.080000
Name: Rad Intensity, dtype: float64